In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
def read_data(file_path):
    # Read your data into a DataFrame
    return pd.read_csv(file_path)

In [ ]:
def prepare_data(data_frame):
    # Target variable
    y = data_frame['Winner']

    # Drop irrelevant features or features with string values
    # The names tend to cause less accuracy when encoded
    X = data_frame.drop(['Winner', 'R_fighter', 'B_fighter'], axis=1)

    # Split the data into train and test sets first
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

    # Split the test set further into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=42)

    # Convert features and labels to PyTorch tensors
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

    return X_train_tensor, X_val_tensor, X_test_tensor, y_train_tensor, y_val_tensor, y_test_tensor


In [ ]:
def create_data_loader(X_train, y_train, batch_size=10):
    # Define DataLoader for training
    train_dataset = TensorDataset(X_train, y_train)
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# creat the class for the model
class FightPredictionModel(nn.Module):

    # constructor with parameters input size
    def __init__(self, input_size):
        super(FightPredictionModel, self).__init__()
        self.fc = nn.Linear(input_size, 1)  # fully connected layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)  # for the ,atrix multiplication
        x = self.sigmoid(x)  # range [0,1]
        return x

In [ ]:
def create_model_and_optimizer(input_size, learning_rate=0.001):
    # Define your neural network model
    model = FightPredictionModel(input_size)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()
    return model, optimizer, criterion

In [ ]:
"""
essentially encapsulates the entire process of training a neural 
network model on a given dataset with the specified optimizer and 
criterion for a certain number of epochs.
"""
def train_model(model, optimizer, criterion, train_loader, num_epochs=1000):
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            loss.backward()    
            optimizer.step()
            epoch_loss += loss.item()
            
        # this is mainly so I can see what is happening and the itteration it is on
        print(f'Epoch [{epoch + 1}/{num_epochs}]')

In [ ]:
def evaluate_model(model, X_test, y_test):
    # Evaluate the final model on the test set
    with torch.no_grad():
        # allows for evaluation
        model.eval()

        # line passing through
        outputs = model(X_test)
        predictions = (outputs >= 0.5).float()
        accuracy = (predictions == y_test.unsqueeze(1)).float().mean()
        print(f'Test Accuracy: {accuracy.item() * 100:.2f}%')


In [ ]:
# Main function
def main():
    file_path = "/Users/reecemilligan/Desktop/AI_Project/ufcdata/ufc-master.csv"
    data_frame = read_data(file_path)
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(data_frame)
    train_loader = create_data_loader(X_train, y_train)
    model, optimizer, criterion = create_model_and_optimizer(input_size=X_train.shape[1], learning_rate=0.0001)
    train_model(model, optimizer, criterion, train_loader, num_epochs=1000)
    evaluate_model(model, X_test, y_test)

if __name__ == "__main__":
    main()